In [1]:
import os
import numpy as np
import pandas as pd
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import dash_bootstrap_components as dbc
import plotly.express as px
import plotly.graph_objects as go
os.getcwd()

'/Users/kristiinajoon/Documents/Projects/KeM/veebirakendus_demo'

In [2]:
# get data

# identify NaN values as empty space, don't import NaN values to dataframe
df = pd.read_csv("energeetika.csv", na_values='', keep_default_na=True) 
# here defo better way, remove unnecessary columns
df = df.drop(df.columns[5:], axis=1)
# remove column 
df = df.drop(df.columns[1], axis=1) 
df = df.drop(df.columns[2], axis=1)
# clean up names
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '') 
df

# prepare data for plotting
df['muu_heide'] = (df['koguheide_maakasutuseta_kt_co2_ekv'] - df['energeetika_kt_co2_ekv'])
df['en_protsent'] = (df['energeetika_kt_co2_ekv'] / df['koguheide_maakasutuseta_kt_co2_ekv'] * 100)
df['muu_protsent'] = (100 - df['en_protsent'])

/var/folders/39/_jkw8gcx2_l0kj6z6ksdt_440000gn/T/ipykernel_3518/1200659283.py:11: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')


In [9]:
# create web application
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.FLATLY])

# define markdown texts
md_top = '''
# Kliimaandmed
  
Interaktiivne rakendus kliimaandmetes orienteerumiseks.
'''
md_dropdown = '''
### Vali graafik:
'''
md_slider = '''
### Vali ajavahemik:
'''
md_footnote = '''
*Selle graafiku koostamisel pole võetud arvesse 
kasvuhoonegaaside heidet maakasutuse muutustest. 
'''

app.layout = dbc.Container([                        # must declare bootstrap theme at top or none of this will work
    # first row for title and subtitle
    dbc.Row([
        html.Div([
            dcc.Markdown(children=md_top, style={'text-align': 'center'})
        ])
    ]),

    # second row for the graph and its interactive elements
    dbc.Row([
        html.Div([
            dcc.Markdown(children=md_dropdown),
            dcc.Dropdown(
                id='dropdown',
                options=[
                    {'label': 'suhteline heide', 'value': 'protsent'},
                    {'label': 'koguheide', 'value': 'kogu'},
                        ],
                value='protsent',
                style={"width": "60%"}),
        ]),
        html.Div(dcc.Graph(id='graph')),  
        html.Div([
            dcc.Markdown(children=md_slider),
            dcc.RangeSlider(
                id='slider',
                min=1990,
                max=2020,
                step=1,
                value=[1990,2020],
                allowCross=False, 
                tooltip={"placement": "top", "always_visible": True})
        ])  
    ]),

    # third row for the footnote
    dbc.Row([
        html.Div(dcc.Markdown(children=md_footnote))
    ])     
])

@app.callback(
    Output('graph', 'figure'),
    [Input(component_id='dropdown', component_property='value'),
    Input(component_id='slider', component_property='value')]
)
def select_graph(plot_type,year_range):
    df_copy = df.copy()                         # make copy of dataframe
    df_copy = df_copy[df_copy['aasta'].between(year_range[0], year_range[1])]
    if plot_type == 'protsent':
        fig = px.bar(df_copy,                   # suhteline heide, graafik protsentides
             x='aasta',
             y=['en_protsent','muu_protsent'],
             barmode='stack',
             color_discrete_map={
             'en_protsent': '#440154',
             'muu_protsent': '#5ec962'}, 
             opacity=0.7)
        return fig
    elif plot_type == 'kogu':
        fig = px.bar(df_copy,                       # absoluutne heide, graafik kt CO2 ekv
             x='aasta',
             y=['energeetika_kt_co2_ekv','muu_heide'],
             barmode='stack',
             color_discrete_map={
             'energeetika_kt_co2_ekv': '#440154',
             'muu_heide': '#5ec962'}, 
             opacity=0.7)
        return fig



app.run_server(debug=True, use_reloader=False)

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
